# mmdet 训练脚本解析
版本号：
- mmdet 2.18.0
- mmcv 1.3.16
- 1.7.0 <= pytorch < 1.9.0

mmdetection 目录结构如下
```
mmdetection
  - mmdet
  - tools
    - train.py
    - dist_train.sh
  - configs
    - faster_rcnn
      - faster_rcnn_r50_fpn_1x_coco.py
  - work_dirs
```
完整的训练脚本参见 `train.py`，此脚本代码结构如下：
```python
import ...
def parse_args():
    ...

def main():
    args = parse_args()

if __name__ == "__main__":
    main()
```
train.py 脚本的启动方式有两种：DP（Data Parallel）启动与DDP（Distributed Data Parallel）启动：

**DP启动**
``` bash
python tools/train.py configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py <more_args>
```
**DDP启动**
```
# 一般会使用 dist_train.sh 脚本，例如单机4卡启动：
# bash dist_train.sh configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py 4
# 最终转换为如下方式启动
python -m torch.distributed.launch --nproc_per_node=4 --master_port=29500 tools/train.py configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py --launcher pytorch <more_args>
```
为了理解这两种启动方式，应该先看看 `parse_args` 函数的具体细节，其源代码及注释如下

In [ ]:
import argparse
from mmcv import DictAction
import os
import warnings

def parse_args():
    parser = argparse.ArgumentParser(description='Train a detector')
    # config文件名
    parser.add_argument('config', help='train config file path')
    # 存储日志及模型checkpoint的目录（相对于启动时的当前路径）
    # 若不传，则默认将这个目录设定为"./work_dirs/faster_rcnn_r50_fpn_1x_coco
    parser.add_argument('--work-dir', help='the dir to save logs and models')
    # checkpoint路径，表示从该checkpoint继续训练，不传则表示从头训练
    parser.add_argument(
        '--resume-from', help='the checkpoint file to resume from')
    # 是否在训练时进行验证，默认进行验证
    parser.add_argument(
        '--no-validate',
        action='store_true',
        help='whether not to evaluate the checkpoint during training')
    # 只能传 --gpus 与 --gpus-ids 参数至多一个，只适用于 DP 启动
    group_gpus = parser.add_mutually_exclusive_group()
    group_gpus.add_argument(
        '--gpus',
        type=int,
        help='number of gpus to use '
        '(only applicable to non-distributed training)')
    group_gpus.add_argument(
        '--gpu-ids',
        type=int,
        nargs='+',
        help='ids of gpus to use '
        '(only applicable to non-distributed training)')
    
    # 随机数种子
    parser.add_argument('--seed', type=int, default=None, help='random seed')
    # 若传入，则表示 torch.backends.cudnn.benchmark=True
    # 这一设定会让pytorch采用确定性算法计算卷积
    parser.add_argument(
        '--deterministic',
        action='store_true',
        help='whether to set deterministic options for CUDNN backend.')
    
    # 在命令行可以通过传入 --options（目前已被弃用） 或 --cfg-options
    # 参数覆盖掉 config 文件（faster_rcnn_r50_fpn_1x_coco.py）中的一些参数
    # 但一般不会传递这两个参数
    parser.add_argument(
        '--options',
        nargs='+',
        action=DictAction,
        help='override some settings in the used config, the key-value pair '
        'in xxx=yyy format will be merged into config file (deprecate), '
        'change to --cfg-options instead.')
    parser.add_argument(
        '--cfg-options',
        nargs='+',
        action=DictAction,
        help='override some settings in the used config, the key-value pair '
        'in xxx=yyy format will be merged into config file. If the value to '
        'be overwritten is a list, it should be like key="[a,b]" or key=a,b '
        'It also allows nested list/tuple values, e.g. key="[(a,b),(c,d)]" '
        'Note that the quotation marks are necessary and that no white space '
        'is allowed.')

    # 只讨论 none 与 pytorch 这两种情况
    # none 对应的是 DP 启动，
    # pytorch 对应的是 DDP 启动
    parser.add_argument(
        '--launcher',
        choices=['none', 'pytorch', 'slurm', 'mpi'],
        default='none',
        help='job launcher')
    
    # DDP 训练时，必须要有这个参数，表示进程/GPU编号，
    # 无需手动传递此参数，由torch.distributed.launch处理
    parser.add_argument('--local_rank', type=int, default=0)
    args = parser.parse_args()
    if 'LOCAL_RANK' not in os.environ:
        os.environ['LOCAL_RANK'] = str(args.local_rank)

    if args.options and args.cfg_options:
        raise ValueError(
            '--options and --cfg-options cannot be both '
            'specified, --options is deprecated in favor of --cfg-options')
    if args.options:
        warnings.warn('--options is deprecated in favor of --cfg-options')
        args.cfg_options = args.options

    return args

接下来，看 `main` 函数的具体内容：这里将 `main` 函数从上至下拆为如下几段进行解析：
- （1）依据传入配置文件路径（configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py）及其他参数得到 `cfg`
- （2）如果采用 DDP 方式训练，则需要进行一些初始化的工作
- （3）配置日志文件，搜集环境信息放入变量 `meta` 中，这个信息最终会被保存在 checkpoint 中
- （4）依据配置构建模型并初始化
- （5）依据配置构建torch.utils.data.Dataset
- （6）训练模型
注意：前5个步骤都比较简单，较为复杂的是第6个步骤，需要再进行展开

第（1）步

In [ ]:
from mmcv import Config


args = parse_args()

cfg = Config.fromfile(args.config)
if args.cfg_options is not None:
    cfg.merge_from_dict(args.cfg_options)

# 这一段使用了importlib的相关函数，动态导入cfg.custom_imports中用字符串表示的模块
# 主要作用是
# import modules from string list.
if cfg.get('custom_imports', None):
    from mmcv.utils import import_modules_from_strings
    import_modules_from_strings(**cfg['custom_imports'])

# set cudnn_benchmark
if cfg.get('cudnn_benchmark', False):
    torch.backends.cudnn.benchmark = True

# work_dir is determined in this priority: CLI > segment in file > filename
if args.work_dir is not None:
    # update configs according to CLI args if args.work_dir is not None
    cfg.work_dir = args.work_dir
elif cfg.get('work_dir', None) is None:
    # use config filename as default work_dir if cfg.work_dir is None
    cfg.work_dir = osp.join('./work_dirs',
                            osp.splitext(osp.basename(args.config))[0])
if args.resume_from is not None:
    cfg.resume_from = args.resume_from

此处需要解释的是

第（2）步

**DP 启动：**
```bash
python tools/train.py configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py \
--work-dir ./faster_rcnn_train gpu-ids 1,3 --seed 2 --deterministic
```

In [ ]:
if args.gpu_ids is not None:
    cfg.gpu_ids = args.gpu_ids
else:
    cfg.gpu_ids = range(1) if args.gpus is None else range(args.gpus)

# init distributed env first, since logger depends on the dist info.
if args.launcher == 'none':
    distributed = False
else:
    distributed = True
    init_dist(args.launcher, **cfg.dist_params)
    # re-set gpu_ids with distributed training mode
    _, world_size = get_dist_info()
    cfg.gpu_ids = range(world_size)

第（3）步

In [ ]:
# create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
# dump config
cfg.dump(osp.join(cfg.work_dir, osp.basename(args.config)))
# init the logger before other steps
timestamp = time.strftime('%Y%m%d_%H%M%S', time.localtime())
log_file = osp.join(cfg.work_dir, f'{timestamp}.log')
logger = get_root_logger(log_file=log_file, log_level=cfg.log_level)

# init the meta dict to record some important information such as
# environment info and seed, which will be logged
meta = dict()
# log env info
env_info_dict = collect_env()
env_info = '\n'.join([(f'{k}: {v}') for k, v in env_info_dict.items()])
dash_line = '-' * 60 + '\n'
logger.info('Environment info:\n' + dash_line + env_info + '\n' +
            dash_line)
meta['env_info'] = env_info
meta['config'] = cfg.pretty_text
# log some basic info
logger.info(f'Distributed training: {distributed}')
logger.info(f'Config:\n{cfg.pretty_text}')

# set random seeds
if args.seed is not None:
    logger.info(f'Set random seed to {args.seed}, '
                f'deterministic: {args.deterministic}')
    set_random_seed(args.seed, deterministic=args.deterministic)
cfg.seed = args.seed
meta['seed'] = args.seed
meta['exp_name'] = osp.basename(args.config)

第（4）步

In [ ]:
model = build_detector(
    cfg.model,
    train_cfg=cfg.get('train_cfg'),
    test_cfg=cfg.get('test_cfg'))
model.init_weights()

第（5）步

In [ ]:
datasets = [build_dataset(cfg.data.train)]
if len(cfg.workflow) == 2:
    val_dataset = copy.deepcopy(cfg.data.val)
    val_dataset.pipeline = cfg.data.train.pipeline
    datasets.append(build_dataset(val_dataset))
if cfg.checkpoint_config is not None:
    # save mmdet version, config file content and class names in
    # checkpoints as meta data
    cfg.checkpoint_config.meta = dict(
        mmdet_version=__version__ + get_git_hash()[:7],
        CLASSES=datasets[0].CLASSES)

第（6）步

In [ ]:
# add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES
train_detector(
    model,
    datasets,
    cfg,
    distributed=distributed,
    validate=(not args.no_validate),
    timestamp=timestamp,
    meta=meta)

In [ ]:
import torch
import os.path as osp
from mmcv.runner import get_dist_info, init_dist
import mmcv
import time
from mmdet.utils import collect_env, get_root_logger

